In [13]:
import os
from pathlib import Path
import xarray as xr
import pandas as pd
import numpy as np

In [14]:
data_path = Path("./data")
haildays_per_month_file = "haildaysM_ch01r.swiss.lv95_20020401000000_20220930000000.nc"
hailsize_per_month_file = "hailsizeM_ch01r.swiss.lv95_20020401000000_20220930000000.nc"

haildays_ds = xr.open_dataset(data_path / haildays_per_month_file)
hailsize_ds = xr.open_dataset(data_path / hailsize_per_month_file)

df_haildays = haildays_ds.to_dataframe()
df_hailsize = hailsize_ds.to_dataframe()

In [15]:
df_haildays

lat        lon  swiss_coordinates haildays
time       chy     chx                                                      
2002-04-01 1065500 2480500  45.730684   5.903697                  0      NaT
                   2481500  45.730859   5.916539                  0      NaT
                   2482500  45.731033   5.929380                  0      NaT
                   2483500  45.731205   5.942221                  0      NaT
                   2484500  45.731376   5.955063                  0      NaT
...                               ...        ...                ...      ...
2022-09-01 1303500 2840500  47.837685  10.651764                  0      NaT
                   2841500  47.837316  10.665111                  0      NaT
                   2842500  47.836946  10.678456                  0      NaT
                   2843500  47.836574  10.691802                  0      NaT
                   2844500  47.836200  10.705148                  0      NaT

[10991610 rows x 4 columns]

In [16]:
df_hailsize

lat        lon  swiss_coordinates  hailsize
time       chy     chx                                                       
2002-04-01 1065500 2480500  45.730684   5.903697                  0       NaN
                   2481500  45.730859   5.916539                  0       NaN
                   2482500  45.731033   5.929380                  0       NaN
                   2483500  45.731205   5.942221                  0       NaN
                   2484500  45.731376   5.955063                  0       NaN
...                               ...        ...                ...       ...
2022-09-01 1303500 2840500  47.837685  10.651764                  0       NaN
                   2841500  47.837316  10.665111                  0       NaN
                   2842500  47.836946  10.678456                  0       NaN
                   2843500  47.836574  10.691802                  0       NaN
                   2844500  47.836200  10.705148                  0       NaN

[10991610 rows x 4 columns]

### merge haildays and hailsize dataframes by multiindex

In [17]:
df_hail = df_haildays.merge(df_hailsize["hailsize"], how="inner", left_index=True, right_index=True)
df_hail

lat        lon  swiss_coordinates haildays  \
time       chy     chx                                                         
2002-04-01 1065500 2480500  45.730684   5.903697                  0      NaT   
                   2481500  45.730859   5.916539                  0      NaT   
                   2482500  45.731033   5.929380                  0      NaT   
                   2483500  45.731205   5.942221                  0      NaT   
                   2484500  45.731376   5.955063                  0      NaT   
...                               ...        ...                ...      ...   
2022-09-01 1303500 2840500  47.837685  10.651764                  0      NaT   
                   2841500  47.837316  10.665111                  0      NaT   
                   2842500  47.836946  10.678456                  0      NaT   
                   2843500  47.836574  10.691802                  0      NaT   
                   2844500  47.836200  10.705148                  0      NaT   

                            hailsize  
time       chy     chx                
2002-04-01 1065500 2480500       NaN  
                   2481500       NaN  
                   2482500       NaN  
                   2483500       NaN  
                   2484500       NaN  
...                              ...  
2022-09-01 1303500 2840500       NaN  
                   2841500       NaN  
                   2842500       NaN  
                   2843500       NaN  
                   2844500       NaN  

[10991610 rows x 5 columns]

### drop datapoints with no `hailsize` or `haildays` values (no measurements available)

In [18]:
df_hail = df_hail[~df_hail.haildays.isna()]
df_hail = df_hail[~df_hail.hailsize.isna()]

In [19]:
df_hail

lat       lon  swiss_coordinates haildays  \
time       chy     chx                                                        
2002-04-01 1182500 2634500  46.792770  7.890503                  0   0 days   
           1187500 2610500  46.838557  7.576278                  0   0 days   
           1192500 2609500  46.883549  7.563273                  0   1 days   
                   2610500  46.883534  7.576393                  0   1 days   
           1203500 2660500  46.979808  8.233808                  0   0 days   
...                               ...       ...                ...      ...   
2022-09-01 1294500 2684500  47.795574  8.566370                  0   0 days   
                   2685500  47.795444  8.579714                  0   0 days   
                   2686500  47.795312  8.593059                  0   0 days   
                   2687500  47.795179  8.606403                  0   0 days   
                   2690500  47.794771  8.646434                  0   0 days   

                            hailsize  
time       chy     chx                
2002-04-01 1182500 2634500       2.0  
           1187500 2610500       2.5  
           1192500 2609500       4.0  
                   2610500       4.0  
           1203500 2660500       2.0  
...                              ...  
2022-09-01 1294500 2684500       0.0  
                   2685500       0.0  
                   2686500       0.0  
                   2687500       0.0  
                   2690500       0.0  

[834005 rows x 5 columns]

### drop `chy` and `chx` level from multiindex 

In [20]:
df_hail.index = df_hail.index.droplevel([1, 2])
df_hail

,lat,lon,swiss_coordinates,haildays,hailsize
time,,,,,
2002-04-01,46.792770,7.890503,0,0 days,2.0
2002-04-01,46.838557,7.576278,0,0 days,2.5
2002-04-01,46.883549,7.563273,0,1 days,4.0
2002-04-01,46.883534,7.576393,0,1 days,4.0
2002-04-01,46.979808,8.233808,0,0 days,2.0
...,...,...,...,...,...
2022-09-01,47.795574,8.566370,0,0 days,0.0
2022-09-01,47.795444,8.579714,0,0 days,0.0
2022-09-01,47.795312,8.593059,0,0 days,0.0


### get `country`, `state` and `district` from `lon` and `lat` data via reverse geocoding

In [21]:
! pip install reverse_geocoder

In [22]:
# offline reverse geocoding
import reverse_geocoder as rg

In [23]:
coordinates = [(x, y) for x, y in zip(df_hail["lat"], df_hail["lon"])]
coordinates[0:10]

[(46.79276983427726, 7.89050251215368),
 (46.83855727394842, 7.576277711422013),
 (46.8835488838835, 7.563273230080763),
 (46.883533848175176, 7.576392784422012),
 (46.97980750155872, 8.233807894747848),
 (47.123720558995366, 8.235943565721179),
 (47.13280560839907, 8.222897425223264),
 (45.99131559201031, 8.993820151171182),
 (45.99113660467614, 9.006722672539098),
 (45.99095613810782, 9.019625105617015)]

In [24]:
reverse_geocoding_results = rg.search(tuple(coordinates))

Loading formatted geocoded file...


In [27]:
reverse_geocoding_results[0:3]

[{'lat': '46.81667',
  'lon': '7.85',
  'name': 'Wald',
  'admin1': 'Bern',
  'admin2': 'Emmental District',
  'cc': 'CH'},
 {'lat': '46.8501',
  'lon': '7.57748',
  'name': 'Niederwichtrach',
  'admin1': 'Bern',
  'admin2': 'Bern-Mittelland District',
  'cc': 'CH'},
 {'lat': '46.87298',
  'lon': '7.561',
  'name': 'Munsingen',
  'admin1': 'Bern',
  'admin2': 'Bern-Mittelland District',
  'cc': 'CH'}]

In [25]:
df_hail["canton"] = np.array([x["admin1"] for x in reverse_geocoding_results])
df_hail["district"] = np.array([x["admin2"] for x in reverse_geocoding_results])
df_hail["country"] = np.array([x["cc"] for x in reverse_geocoding_results])

In [26]:
df_hail.head()

,lat,lon,swiss_coordinates,haildays,hailsize,canton,district,country
time,,,,,,,,
2002-04-01,46.792770,7.890503,0,0 days,2.0,Bern,Emmental District,CH
2002-04-01,46.838557,7.576278,0,0 days,2.5,Bern,Bern-Mittelland District,CH
2002-04-01,46.883549,7.563273,0,1 days,4.0,Bern,Bern-Mittelland District,CH
2002-04-01,46.883534,7.576393,0,1 days,4.0,Bern,Bern-Mittelland District,CH
2002-04-01,46.979808,8.233808,0,0 days,2.0,Obwalden,Obwalden,CH


### save interim data to CSV

In [28]:
df_hail.to_csv(data_path / "interim_merged_haildata.csv", sep=";", index=False)